# Imports

In [73]:
import pandas as pd
import numpy as np
import csv
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, log_loss
from sklearn.metrics import mean_absolute_error
import pickle as pck
import time
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

# Load in dataset

In [89]:
df = pd.read_csv("../input/input.csv")

/var/folders/3m/1391qbxd3jg3m15k2d19w9qw0000gn/T/ipykernel_25187/3970109353.py:1: DtypeWarning: Columns (8,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../input/input.csv")


In [90]:
df.head()

,TEAM,RECORD_PCT,PPG,FG%,FGAPG,OBPI,DBPI,SOR,SCORE,OPP_TEAM,OPP_RECORD_PCT,OPP_PPG,OPP_FG%,OPP_FGAPG,OPP_OBPI,OPP_DBPI,OPP_SOR,OPP_SCORE,WINNER
0,Austin Peay Governors,0.686,74.6,45.1,54.647059,2.9,-1.8,135,54,Texas Longhorns,0.816,75.6,45.3,59.970588,9.4,4.3,7,74,1
1,Texas Longhorns,0.816,75.6,45.3,59.970588,9.4,4.3,7,74,Austin Peay Governors,0.686,74.6,45.1,54.647059,2.9,-1.8,135,54,0
2,Austin Peay Governors,0.686,74.6,45.1,54.647059,2.9,-1.8,135,67,Vanderbilt Commodores,0.765,80.6,46.7,58.606061,10.0,-1.6,26,81,1
3,Vanderbilt Commodores,0.765,80.6,46.7,58.606061,10.0,-1.6,26,81,Austin Peay Governors,0.686,74.6,45.1,54.647059,2.9,-1.8,135,67,0
4,Austin Peay Governors,0.686,74.6,45.1,54.647059,2.9,-1.8,135,71,Belmont Bruins,0.735,80.1,45.2,61.181818,6.8,-4.2,106,56,0


In [91]:
df = df.dropna()

# Predict Score

In [92]:
df.SCORE = df.SCORE.apply(pd.to_numeric, errors="coerce")
df.OPP_SCORE = df.OPP_SCORE.apply(pd.to_numeric, errors="coerce")

/var/folders/3m/1391qbxd3jg3m15k2d19w9qw0000gn/T/ipykernel_25187/1678557736.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.SCORE = df.SCORE.apply(pd.to_numeric, errors="coerce")
/var/folders/3m/1391qbxd3jg3m15k2d19w9qw0000gn/T/ipykernel_25187/1678557736.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.OPP_SCORE = df.OPP_SCORE.apply(pd.to_numeric, errors="coerce")


In [93]:
df = df.drop(columns=[
      "OPP_SCORE", 
      "SCORE",
      "TEAM",
      "OPP_TEAM"])
df = df.dropna()
y = df.WINNER
x = df.drop(columns=["WINNER"])

In [94]:
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.25, random_state=31523)

In [95]:
start = time.time()

rf = RandomForestClassifier(
    n_estimators=100,
    min_samples_leaf=5,
    max_features='sqrt',
    max_depth=25,
    bootstrap=True)

rf.fit(train_x, train_y)

end = time.time()

In [96]:
### Random Forest
rf_pred = rf.predict(test_x)
rf_acc = accuracy_score(test_y, rf_pred)
rf_mae = mean_absolute_error(test_y, rf_pred)
print(f"{'Random Forest MAE:':<20} ==> {rf_mae}")
rf_predprob = rf.predict_proba(test_x)
print(f"{'Random Forest':<20} ==> {round(rf_acc * 100, 2)}")

Random Forest MAE:   ==> 0.27284058332294653
Random Forest        ==> 72.72


In [97]:
### Logistic Regression
log = LogisticRegression()
log.fit(train_x, train_y)

/Users/regbert/rf/application/venv38/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [98]:
log_pred = log.predict(test_x)
log_acc = accuracy_score(test_y, log_pred)
log_mae = mean_absolute_error(test_y, log_pred)
print(f"{'Logistic Regression MAE:':<20} ==> {log_mae}")
log_predprob = log.predict_proba(test_x)
print(f"{'Logistic Regression':<20} ==> {round(log_acc * 100, 2)}")

Logistic Regression MAE: ==> 0.24507042253521127
Logistic Regression  ==> 75.49


In [99]:
### ANN
ann = MLPClassifier(hidden_layer_sizes = (10,50,10))
ann.fit(train_x, train_y)

MLPClassifier(hidden_layer_sizes=(10, 50, 10))

In [100]:
### ANN
ann_pred = ann.predict(test_x)
ann_acc = accuracy_score(test_y, ann_pred)
ann_mae = mean_absolute_error(test_y, ann_pred)
print(f"{'Neural Net MAE:':<20} ==> {ann_mae}")
ann_predprob = ann.predict_proba(test_x)
print(f"{'Neural Net':<20} ==> {round(ann_acc * 100, 2)}")

Neural Net MAE:      ==> 0.24459678424529477
Neural Net           ==> 75.54


# Selected Model

In [101]:
with open("ann_winner.pck", 'wb') as f:
    pck.dump(ann, f)

In [102]:
test_x

,RECORD_PCT,PPG,FG%,FGAPG,OBPI,DBPI,SOR,OPP_RECORD_PCT,OPP_PPG,OPP_FG%,OPP_FGAPG,OPP_OBPI,OPP_DBPI,OPP_SOR
53540,0.657,77.0,45.0,59.000000,6.1,-3.7,111,0.657,66.9,43.3,54.906250,-0.1,3.0,114
100048,0.714,71.2,44.1,58.580645,-1.7,5.1,85,0.865,76.3,47.6,55.088235,8.8,5.9,7
130128,0.656,68.7,49.7,50.500000,1.3,2.9,117,0.625,75.5,46.9,56.031250,3.7,-0.5,98
20855,0.367,67.7,43.3,53.300000,-1.3,-2.7,246,0.500,61.7,43.7,51.468750,-5.6,5.3,173
104210,0.267,61.5,44.9,51.400000,-6.0,-2.0,320,0.743,76.1,48.1,55.941176,2.8,2.4,83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40649,0.344,62.2,40.0,54.093750,-5.0,2.7,207,0.784,71.5,46.2,56.181818,8.6,6.5,12
102076,0.516,72.3,44.1,59.600000,-2.6,0.9,194,0.629,80.5,45.5,61.500000,4.3,-3.2,113
121288,0.471,69.6,42.3,54.843750,-4.3,-2.4,265,0.276,61.2,41.5,49.724138,-6.1,-2.7,308
157031,0.167,69.6,43.1,58.366667,-4.8,-9.0,327,0.194,70.3,40.5,58.903226,-3.8,-9.2,334
